In [ ]:
import torch
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from torchvision import datasets, transforms, models
from torchvision.models import ResNet101_Weights
import torch.nn as nn
# Hyperparameters and paths (modify as necessary)
dropout_rate = 0.6
batch_size = 32
num_classes = 2 # Modify based on your problem
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Transforms for test dataset (same as validation)
val_transforms = transforms.Compose([
transforms.Resize(256),
transforms.CenterCrop(224),
transforms.ToTensor(),
transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
# Load the test dataset
test_dataset = datasets.ImageFolder(root='/kaggle/input/audiospec/output_spectrograms/output_spectrograms', transform=val_transforms)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
# Model: Initialize the same model architecture
model = models.resnet101(weights=ResNet101_Weights.DEFAULT)
num_features = model.fc.in_features
model.fc = nn.Sequential(
nn.Dropout(dropout_rate),
nn.Linear(num_features, num_classes)
)
# Load the saved model weights
model_path = '/kaggle/input/res101haha/pytorch/default/1/resnet101_model_weights.pth' # Update this with the actual path
model.load_state_dict(torch.load(model_path)) # Load the model weights
model.to(device) # Send the model to the appropriate device (GPU or CPU)
# Set the model to evaluation mode
model.eval()
# Function to evaluate on the test set and compute metrics
def evaluate_on_test_set(model, test_loader):
all_labels = []
all_predictions = []
with torch.no_grad(): # Disable gradient calculation for inference
for images, labels in test_loader:
images, labels = images.to(device), labels.to(device)
outputs = model(images)
_, predicted = torch.max(outputs.data, 1) # Get predictions
# Store true labels and predictions
all_labels.extend(labels.cpu().numpy())
all_predictions.extend(predicted.cpu().numpy())
# Calculate precision, recall, F1 score, and accuracy
precision = precision_score(all_labels, all_predictions, average='macro')
recall = recall_score(all_labels, all_predictions, average='macro')
f1 = f1_score(all_labels, all_predictions, average='macro')
accuracy = accuracy_score(all_labels, all_predictions)
print(f'Precision (Macro): {precision:.4f}')
print(f'Recall (Macro): {recall:.4f}')
print(f'F1 Score (Macro): {f1:.4f}')
print(f'Accuracy: {accuracy:.4f}')
return all_labels, all_predictions
# Function to plot confusion matrix
def plot_confusion_matrix(all_labels, all_predictions, class_names):
cm = confusion_matrix(all_labels, all_predictions)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()
# Run the inference on the test set
all_test_labels, all_test_predictions = evaluate_on_test_set(model, test_loader)
# Plot confusion matrix
plot_confusion_matrix(all_test_labels, all_test_predictions, class_names=test_dataset.classes)